In [1]:
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import json

In [2]:
from keras.layers.core import Dense, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
def the_model():
    model = Sequential()
    model.add(Convolution2D(24, 5, 5, border_mode='same', input_shape=(160, 320, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation(activation='relu'))
    model.add(Convolution2D(36, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation(activation='relu'))
    model.add(Convolution2D(48, 5, 5, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation(activation='relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation(activation='relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Flatten())
    model.add(Activation(activation='relu'))
    model.add(Dense(1164, name='Dense1'))
    model.add(Activation(activation='relu'))
    model.add(Dense(100, name='Dense2'))
    model.add(Activation(activation='relu'))
    model.add(Dense(50, name='Dense3'))
    model.add(Activation(activation='relu'))
    model.add(Dense(10, name='Dense4'))
    model.add(Activation(activation='relu'))
    model.add(Dense(1, name='out1'))

    return model

In [4]:
def the_simpler_model():
    model = Sequential()
    model.add(Flatten(input_shape=(160, 320, 3)))
    model.add(Dense(1000))
    model.add(Activation(activation='relu'))
    model.add(Dense(500))
    model.add(Activation(activation='relu'))
    model.add(Dense(100))
    model.add(Activation(activation='relu'))
    model.add(Dense(10))
    model.add(Activation(activation='relu'))
    model.add(Dense((10, 1)))
    
    return model

In [5]:
def save_parameters(model):
    model.save_weights('model.h5')
    json_file = open('model.json', mode='w')
    json.dump(model.to_json(), json_file)

In [ ]:
log = pd.read_csv('./data/driving_log.csv')

_X_train = log['center']
y_train = log['steering']
del log
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(_X_train, y_train, test_size=0.1)

def generator(X, Y):
    while 1:
        for (x, y) in zip(X, Y):
            path, angle = (x, y)
            angle = np.reshape(angle, [1])
            image = cv2.imread('./data/' + path)
            image = np.reshape(image, (1, 160, 320, 3))
            yield image, angle

In [7]:
batches = 250
epochs = 500

In [ ]:
model = the_model()
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
print(y_train.shape)
model.fit_generator(generator(X_train, y_train), 
                    samples_per_epoch= X_train.shape[0],
                    nb_epoch=epochs,
                    verbose=1,
                    validation_data=generator(X_test, y_test),
                    nb_val_samples=X_test.shape[0])

save_parameters(model)

(7232,)
Epoch 1/500
 233/7232 [..............................] - ETA: 378s - loss: 0.0435 - acc: 0.4850

In [248]:
model.evaluate(X_valid, y_valid, batch_size=32, verbose=2)

NameError: name 'X_valid' is not defined

['loss']
